In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save

from transformations import chargenet_trafo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data = Data()

In [4]:
charge_input = tf.keras.Input(shape=(1,))
params_input = tf.keras.Input(shape=(len(data.labels),))
t = chargenet_trafo(labels=data.labels)

h = t(charge_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

charge_net = tf.keras.Model(inputs=[charge_input, params_input], outputs=outputs)
charge_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
chargenet_trafo (chargenet_traf (None, 9)            0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           320         chargenet_trafo[0][0]            
______________________________________________________________________________________________

In [5]:
optimizer = tf.keras.optimizers.Adam(0.00001)
charge_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [6]:
train_id = 'ChargeNet_' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [7]:
callbacks = []
callbacks.append(Save(save_every=25, path_template='models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [9]:
train_data, test_data = data.get_chargenet_data(train_batch_size=128,
                                             test_batch_size=128,
                                            )

In [ ]:
hist = charge_net.fit(train_data, epochs=10000, verbose=2, callbacks=callbacks, validation_data=test_data)

Epoch 1/10000
6651/6651 - 23s - loss: 0.6901 - val_loss: 0.0000e+00
Epoch 2/10000
6651/6651 - 18s - loss: 0.6502 - val_loss: 0.6339
Epoch 3/10000
6651/6651 - 19s - loss: 0.6402 - val_loss: 0.6277
Epoch 4/10000
6651/6651 - 20s - loss: 0.6347 - val_loss: 0.6255
Epoch 5/10000
6651/6651 - 20s - loss: 0.6311 - val_loss: 0.6241
Epoch 6/10000
6651/6651 - 19s - loss: 0.6284 - val_loss: 0.6235
Epoch 7/10000
6651/6651 - 19s - loss: 0.6261 - val_loss: 0.6185
Epoch 8/10000
6651/6651 - 19s - loss: 0.6245 - val_loss: 0.6194
Epoch 9/10000
6651/6651 - 19s - loss: 0.6229 - val_loss: 0.6256
Epoch 10/10000
6651/6651 - 19s - loss: 0.6217 - val_loss: 0.6171
Epoch 11/10000
6651/6651 - 20s - loss: 0.6204 - val_loss: 0.6157
Epoch 12/10000
6651/6651 - 20s - loss: 0.6194 - val_loss: 0.6204
Epoch 13/10000
